# Equivariant Neural Field (ENF) Training

This notebook implements training for an Equivariant Neural Field model. ENFs are a type of neural field that respects certain symmetries in the data, making them particularly effective for image representation tasks.

## Imports and Environment Setup

In [26]:
import os 
import math
import logging
from pathlib import Path

import ml_collections
import torch
import jax
import jax.numpy as jnp
import optax
import orbax.checkpoint as ocp
import matplotlib.pyplot as plt
from tqdm import tqdm

import numpy as np
import torchvision
import torch.utils.data as data

import jax
import jax.numpy as jnp
from flax import linen as nn

from typing import Callable, Union, Any, Sequence

# Set environment to use GPU
os.environ['JAX_PLATFORM_NAME'] = 'gpu'

## Configuration

Define the configuration for the model, dataset, and training parameters.

In [27]:
# Define config
config = ml_collections.ConfigDict()
config.seed = 68
config.debug = False

# Define the ENF model parameters
config.enf = ml_collections.ConfigDict()
config.enf.num_in = 2           # Images are 2D
config.enf.num_out = 3          # RGB images = 3 channels, grayscale = 1
config.enf.num_hidden = 128
config.enf.num_heads = 3
config.enf.att_dim = 128
config.enf.num_latents = 4
config.enf.latent_dim = 32
config.enf.freq_multiplier_query = 1.0 
config.enf.freq_multiplier_value = 2.0 
config.enf.k_nearest = 4
config.enf.bi_invariant = "translation"  # Choose between translation and roto_translation_2d

# Dataset config
config.dataset = ml_collections.ConfigDict()
config.dataset.path = "./data"
config.dataset.name = "FIGURE"           # Choose between cifar10 and FIGURE
config.dataset.num_signals_train = 1000
config.dataset.num_signals_test = 1000
config.dataset.batch_size = 32
config.dataset.num_workers = 8

# Specific FIGURE dataset parameters
config.dataset.figure_type = "FIGURE-Shape-B"  # Choose between FIGURE-Shape-B and FIGURE-Shape-CB
config.dataset.swap_bias = False               # Mainly interesting for down-stream tasks
config.dataset.color_consistency = 0.9

# Optimizer config
config.optim = ml_collections.ConfigDict()
config.optim.lr_enf = 5e-4
config.optim.inner_lr_c = 15.
config.optim.inner_lr_p = 0.
config.optim.inner_lr_g = 0.
config.optim.inner_steps = 3

# Training config
config.train = ml_collections.ConfigDict()
config.train.num_epochs = 5000
config.train.log_interval = 100



# Some utility functions

In [28]:
def image_to_numpy(image):
    return np.array(image, dtype=np.float32) / 255


def numpy_collate(batch: Union[np.ndarray, Sequence[Any], Any]):
    """
    TODO: this might be a repeat, maybe it's ok to make it special for shapes, but needs a check
    Collate function for numpy arrays.

    This function acts as replacement to the standard PyTorch-tensor collate function in PyTorch DataLoader.

    Args:
        batch: Batch of data. Can be a numpy array, a list of numpy arrays, or nested lists of numpy arrays.

    Returns:
        Batch of data as (potential list or tuple of) numpy array(s).
    """
    if isinstance(batch, np.ndarray):
        return batch
    elif isinstance(batch[0], np.ndarray):
        return np.stack(batch)
    elif isinstance(batch[0], (tuple, list)):
        transposed = zip(*batch)
        return [numpy_collate(samples) for samples in transposed]
    else:
        return np.array(batch)
    

def get_dataset(dataset_cfg):
    """ Function which gets the dataset based on the dataset config. For now it only supports CIFAR10 and the FIGURE dataset."""

    if dataset_cfg.name == "cifar10":
        from torchvision.datasets import CIFAR10

        transform = torchvision.transforms.Compose([image_to_numpy])
        train_dset = torchvision.datasets.CIFAR10(root=dataset_cfg.path, train=1, transform=transform, download=1)
        test_dset = torchvision.datasets.CIFAR10(root=dataset_cfg.path, train=0, transform=transform, download=1)
    elif dataset_cfg.name == "FIGURE":
        from datasets.figure_dset import FigureDataset

        transform = torchvision.transforms.Compose([image_to_numpy, lambda x: x.transpose(1, 2, 0)])
        train_dset = FigureDataset("FIGURE-Shape-B", split="train", color_consistency=0.9, transform=transform, download=True)  

        if dataset_cfg.swap_bias:
            test_dset = FigureDataset("FIGURE-Shape-CB", split="test-bias", color_consistency=0.9, transform=transform, download=True)
        else:
            test_dset = FigureDataset("FIGURE-Shape-B", split="test", color_consistency=0.9, transform=transform, download=True)
    else:
        raise ValueError(f"Unknown dataset name: {dataset_cfg.name}")
    
    return train_dset, test_dset


def get_dataloader(dataset_cfg, shuffle_train=True):
    """ Function which gets the dataloader based on the dataset config. """

    train_dset, test_dset = get_dataset(dataset_cfg)
    
    if dataset_cfg.num_signals_train != -1:
        train_dset = data.Subset(train_dset, np.arange(0, dataset_cfg.num_signals_train))
    if dataset_cfg.num_signals_test != -1:
        test_dset = data.Subset(test_dset, np.arange(0, dataset_cfg.num_signals_test))

    train_loader = data.DataLoader(
        train_dset,
        batch_size=dataset_cfg.batch_size,
        shuffle=shuffle_train,
        num_workers=dataset_cfg.num_workers,
        collate_fn=numpy_collate,
        persistent_workers=False,
        drop_last=True
    )

    test_loader = data.DataLoader(
        test_dset,
        batch_size=dataset_cfg.batch_size,
        shuffle=False,
        num_workers=dataset_cfg.num_workers,
        collate_fn=numpy_collate,
        persistent_workers=False,
        drop_last=True
    )
    return train_loader, test_loader


## Model Definition
### Positional embedding

The positional embedding is a key component that transforms spatial coordinates into a higher-dimensional representation. The mathematical formula is:

1. First, scale the input coordinates: $\pi(x + 1)$
2. Project to a lower dimension with random weights: $e = W \cdot \pi(x + 1)$ where $W \sim \mathcal{N}(0, \text{freq})$
3. Apply sinusoidal encoding: $\sin([x, e, e + \pi/2])$
4. Project to final embedding dimension: $\text{PosEmb}(x) = W_{\text{out}} \cdot \sin([x, e, e + \pi/2])$

This embedding helps the model capture spatial relationships at different frequencies.

In [29]:
class PosEmb(nn.Module):
    embedding_dim: int
    freq: float

    @nn.compact
    def __call__(self, coords):
        emb = nn.Dense(self.embedding_dim // 2, kernel_init=nn.initializers.normal(self.freq), use_bias=False)(
            jnp.pi * (coords + 1))
        return nn.Dense(self.embedding_dim)(jnp.sin(jnp.concatenate([coords, emb, emb + jnp.pi / 2.0], axis=-1)))



### Visualize the positional embedding

In [30]:

# Initialize the positional embedding for different frequencies
freqs = [1.0, 5.0, 20.0]
posemb = [PosEmb(128, freq) for freq in freqs]

# Create a grid of coordinates
grid = jnp.stack(jnp.meshgrid(jnp.linspace(-1, 1, 100), jnp.linspace(-1, 1, 100)), axis=-1)
grid = jnp.reshape(grid, (-1, 2))
grid = jnp.repeat(grid[None, ...], 1, axis=0)
params = [posemb[i].init(jax.random.PRNGKey(i), grid) for i in range(len(posemb))]
out = [posemb[i].apply(params[i], grid) for i in range(len(posemb))]

# Plot the output for each frequency
plt.figure(figsize=(15, 4))
for i in range(len(posemb)):
    plt.subplot(1, 3, i+1)
    plt.imshow(jnp.mean(out[i][0].reshape(100, 100, -1), axis=-1)) # average over the embedding dimension
    plt.colorbar()
    plt.title(f"Frequency: {freqs[i]}")
plt.tight_layout()
plt.show()


## Define the bi-invariant functions

Bi-invariant functions are crucial for ensuring equivariance in neural fields. They transform input coordinates and latent poses into representations that respect certain symmetries:

1. **Translation Bi-invariant**:
   - Formula: $\text{BI}_{\text{trans}}(x, p) = x - p$
   - This creates a representation that is invariant to global translations of both input coordinates and latent poses.

2. **Roto-Translation Bi-invariant (2D)**:
   - For input coordinates $x \in \mathbb{R}^2$ and latent poses $p = (p_{\text{pos}}, \theta) \in \mathbb{R}^2 \times \mathbb{R}$:
   - First compute relative position: $\Delta x = x - p_{\text{pos}}$
   - Then rotate by $-\theta$ to get invariant representation:
     - $\text{BI}_{\text{roto-trans}}(x, p)_1 = \Delta x_1 \cos(\theta) + \Delta x_2 \sin(\theta)$
     - $\text{BI}_{\text{roto-trans}}(x, p)_2 = -\Delta x_1 \sin(\theta) + \Delta x_2 \cos(\theta)$
   - This creates a representation that is invariant to both translations and rotations.

These bi-invariants ensure that the neural field's predictions remain consistent under the corresponding transformations of the input space.

In [18]:
def get_bi_invariant(name: str):
    """ Get the bi-invariant function by name. Currently supports 'translation' and 'roto_translation_2d'"""
    
    if name == "translation":
        return TranslationBI()
    elif name == "roto_translation_2d":
        return RotoTranslationBI2D()
    else:
        raise ValueError(f"Bi-invariant {name} not found.")


class TranslationBI:   
    def __call__(self, x: jnp.ndarray, p: jnp.ndarray) -> jnp.ndarray:
        """Compute the translation equivariant bi-invariant for ND data.
        
        Args:
            x: The input data. Shape (batch_size, num_coords, coord_dim).
            p: The latent poses. Shape (batch_size, num_latents, coord_dim).
        """
        return x[:, :, None, :] - p[:, None, :, :]


class RotoTranslationBI2D:
    def __call__(self, x: jnp.ndarray, p: jnp.ndarray) -> jnp.ndarray:
        """ Compute roto-translation bi-invariant for 2D data. This assumes the input coordinates are
        2D, i.e. do not include the orientation. The latents include orientation angle theta.
        
        Args:
            x: The input data. Shape (batch_size, num_coords, pos_dim (2)).
            p: The latent poses. Shape (batch_size, num_latents, pos_dim (2) + theta (1)).
        """
        # Compute the relative position between the input and the latent poses
        rel_pos = x[:, :, None, :2] - p[:, None, :, :2]

        # Get the orientation angle theta and convert to [cos(θ), sin(θ)]
        theta = p[:, None, :, 2:]
        cos_theta = jnp.cos(theta)
        sin_theta = jnp.sin(theta)

        # Compute the relative orientation between the input and the latent poses
        invariant1 = rel_pos[..., 0] * cos_theta + rel_pos[..., 1] * sin_theta
        invariant2 = -rel_pos[..., 0] * sin_theta + rel_pos[..., 1] * cos_theta
        return jnp.stack([invariant1, invariant2], axis=-1)


### Equivariant Neural Field

In [19]:
class EquivariantNeuralField(nn.Module):
    """ Equivariant cross attention network for the latent points, conditioned on the poses.

    Args:
        num_hidden (int): The number of hidden units.
        num_heads (int): The number of attention heads.
        num_out (int): The number of output coordinates.
        emb_freq_q (float): The frequency of the positional embedding for the query.
        emb_freq_v (float): The frequency of the positional embedding for the value function.
        nearest_k (int): The number of nearest latents to consider.
        bi_invariant (Callable): The bi-invariant function to use.
    """
    num_hidden: int
    att_dim: int
    num_heads: int
    num_out: int
    emb_freq_q: float
    emb_freq_v: float
    nearest_k: int
    bi_invariant: Callable

    def setup(self):
        # Positional embedding that takes in relative positions.
        self.pos_emb_q = nn.Sequential([PosEmb(self.num_hidden, self.emb_freq_q), nn.Dense(self.num_heads * self.att_dim)])
        self.pos_emb_v = nn.Sequential([PosEmb(self.num_hidden, self.emb_freq_v), nn.Dense(2 * self.num_hidden)])

        # Query, key, value functions.
        self.W_k = nn.Dense(self.num_heads * self.att_dim)
        self.W_v = nn.Dense(self.num_hidden)

        # Value bi-linear conditioning function.
        self.v = nn.Sequential([
            nn.Dense(self.num_hidden),
            nn.gelu,
            nn.Dense(self.num_heads * self.num_hidden)])

        # Output layer.
        self.W_out = nn.Dense(self.num_out)

    def __call__(self, x, p, c, g):
        """ Apply equivariant cross attention.

        Args:
            x (jax.numpy.ndarray): The input coordinates. Shape (batch_size, num_coords, coord_dim).
            p (jax.numpy.ndarray): The latent poses. Shape (batch_size, num_latents, coord_dim).
            c (jax.numpy.ndarray): The latent context vectors. Shape (batch_size, num_latents, latent_dim).
            g (jax.numpy.ndarray): The window size for the gaussian window. Shape (batch_size, num_latents, 1).
        """
        # Calculate bi-invariants between input coordinates and latents
        bi_inv = self.bi_invariant(x, p)
        
        # Calculate distances based on bi-invariant magnitude
        zx_mag = jnp.sum(bi_inv ** 2, axis=-1)
        nearest_z = jnp.argsort(zx_mag, axis=-1)[:, :, :self.nearest_k, None]

        # Restrict the bi-invariants and context vectors to the nearest latents
        zx_mag = jnp.take_along_axis(zx_mag[..., None], nearest_z, axis=2)
        bi_inv = jnp.take_along_axis(bi_inv, nearest_z, axis=2)
        k = jnp.take_along_axis(self.W_k(c)[:, None, :, :], nearest_z, axis=2)
        v = jnp.take_along_axis(self.W_v(c)[:, None, :, :], nearest_z, axis=2)
        g = jnp.take_along_axis(g[:, None, :, :], nearest_z, axis=2)

        # Apply bi-invariant embedding for the query transform and conditioning of the value transform
        q = self.pos_emb_q(bi_inv)
        b_v, g_v = jnp.split(self.pos_emb_v(bi_inv), 2, axis=-1)
        v = self.v(v * (1 + b_v) + g_v)

        # Reshape to separate the heads
        q = q.reshape(q.shape[:-1] + (self.num_heads, -1))
        k = k.reshape(k.shape[:-1] + (self.num_heads, -1))
        v = v.reshape(v.shape[:-1] + (self.num_heads, -1))

        # Calculate the attention weights, apply gaussian mask based on bi-invariant magnitude, broadcasting over heads.
        att_logits = ((q * k).sum(axis=-1, keepdims=True) / self.att_dim) - ((1 / g ** 2) * zx_mag)[..., None, :]
        att = jax.nn.softmax(att_logits, axis=2)

        # Attend the values to the queries and keys.
        y = (att * v).sum(axis=2)

        # Combine the heads and apply the output layer.
        y = y.reshape(y.shape[:-2] + (-1,))
        return self.W_out(y)


## Model Initialization

Load the dataset, create coordinate grid, and initialize the ENF model.

In [20]:
# Load dataset, get sample image, create corresponding coordinates
train_dloader, test_dloader = get_dataloader(config.dataset)
sample = next(iter(train_dloader))
img_shape = sample[0].shape[1:]

# Random key for JAX operations
key = jax.random.PRNGKey(55)

# Create coordinate grid
x = jnp.stack(jnp.meshgrid(jnp.linspace(-1, 1, img_shape[0]), jnp.linspace(-1, 1, img_shape[1])), axis=-1)
x = jnp.reshape(x, (-1, 2))
x = jnp.repeat(x[None, ...], config.dataset.batch_size, axis=0)

# Define the model
bi_inv = get_bi_invariant(config.enf.bi_invariant)
model = EquivariantNeuralField(
    num_hidden=config.enf.num_hidden,
    att_dim=config.enf.att_dim,
    num_heads=config.enf.num_heads,
    num_out=config.enf.num_out,
    emb_freq_q=config.enf.freq_multiplier_query,
    emb_freq_v=config.enf.freq_multiplier_value,
    nearest_k=config.enf.k_nearest,
    bi_invariant=bi_inv,
)

/gpfs/home1/rvalperga/miniforge3/envs/enf/lib/python3.11/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/gpfs/home1/rvalperga/miniforge3/envs/enf/lib/python3.11/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/tmp/ipykernel_2702159/4013305820.py:2: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  return np.array(image, dtype=np.float32) / 255
/tmp/ipykernel_2702159/4013305820.py:2: DeprecationWarning: __array_

## Initialize Model Parameters

In JAX, we need to initialize model parameters by passing dummy inputs through the model so it can infer parameter shapes for JIT compilation.

In [21]:
# Create dummy latents for model initialization
d_p = jnp.ones((config.dataset.batch_size, config.enf.num_latents, 2))                        # poses
d_c = jnp.ones((config.dataset.batch_size, config.enf.num_latents, config.enf.latent_dim))    # context vectors
d_g = jnp.ones((config.dataset.batch_size, config.enf.num_latents, 1))                        # gaussian window parameter

# Initialize the model parameters
enf_params = model.init(key, x, d_p, d_c, d_g)

# Define optimizer for the ENF backbone
enf_optimizer = optax.adam(learning_rate=config.optim.lr_enf)
enf_opt_state = enf_optimizer.init(enf_params)

## Define Training Functions

Define the inner loop and outer step functions for training. The inner loop optimizes the latent variables, while the outer step updates the model parameters.

In [22]:
@jax.jit
def inner_loop(enf_params, x_i, y_i, key):
    # Sample poses
    if config.enf.num_latents == 1:
        poses = jnp.zeros((1, config.enf.num_latents, 2))
    else: # Initialize poses in a grid
        lims = 1 - 1 / math.sqrt(config.enf.num_latents)
        poses = jnp.stack(jnp.meshgrid(jnp.linspace(-lims, lims, int(math.sqrt(config.enf.num_latents))),
                                       jnp.linspace(-lims, lims, int(math.sqrt(config.enf.num_latents)))), axis=-1)
        poses = jnp.reshape(poses, (1, -1, 2))
    poses = poses.repeat(config.dataset.batch_size, axis=0)

    # Add some noise to the poses
    poses = poses + jax.random.normal(key, poses.shape) * 0.1 / jnp.sqrt(config.enf.num_latents)

    # Initialize values for the poses, context and window
    c = jnp.ones((x_i.shape[0], config.enf.num_latents, config.enf.latent_dim)) / config.enf.latent_dim  # context vectors
    g = jnp.ones((x_i.shape[0], config.enf.num_latents, 1)) * 2 / jnp.sqrt(config.enf.num_latents)  # gaussian window parameter

    def mse_loss(z, x_i, y_i):
        out = model.apply(enf_params, x_i, *z)
        return jnp.sum(jnp.mean((out - y_i) ** 2, axis=(1, 2)), axis=0)

    loss, grads = jax.value_and_grad(mse_loss)((poses, c, g), x_i, y_i)

    # Update the latent features
    c = c - config.optim.inner_lr_c * grads[1]

    # Update the poses if specific lr > 0
    if config.optim.inner_lr_p > 0:
        poses = poses - config.optim.inner_lr_p * grads[0]
    if config.optim.inner_lr_g > 0:
        g = g - config.optim.inner_lr_g * grads[2]

    # Return loss with resulting latents
    return mse_loss((poses, c, g), x_i, y_i), (poses, c, g)

@jax.jit
def outer_step(x_i, y_i, enf_params, enf_opt_state, key):
    # Split key
    key, new_key = jax.random.split(key)

    # Perform inner loop optimization
    (loss, _), grads = jax.value_and_grad(inner_loop, has_aux=True)(enf_params, x_i, y_i, key)

    # Update the ENF backbone
    enf_grads, enf_opt_state = enf_optimizer.update(grads, enf_opt_state)
    enf_params = optax.apply_updates(enf_params, enf_grads)

    # Return updated parameters and new key
    return loss, enf_params, enf_opt_state, new_key

## Training Loop

Execute the training loop, periodically logging results and saving checkpoints.

In [24]:
# Training loop
glob_step, lowest_loss = 0, jnp.inf
for epoch in range(config.train.num_epochs):
    epoch_loss = []
    for i, batch in tqdm(enumerate(train_dloader), total=len(train_dloader), desc=f'Epoch {epoch}'):
        # Unpack batch, flatten img
        img = batch[0]
        y = jnp.reshape(img, (img.shape[0], -1, img.shape[-1]))

        # Perform outer loop optimization
        loss, enf_params, enf_opt_state, key = outer_step(
            x, y, enf_params, enf_opt_state, key)

        epoch_loss.append(loss)
        glob_step += 1

        if glob_step % config.train.log_interval == 0:
            # Reconstruct and log an image, perform inner loop
            _, (p_b, c, g) = inner_loop(enf_params, x, y, key)

            # Reconstruct image
            img_r = model.apply(enf_params, x, p_b, c, g)[0]

            # Min max normalization
            min_val = jnp.min(img[0])
            max_val = jnp.max(img[0])

            img_r = (img_r - jnp.min(img_r)) / (jnp.max(img_r) - jnp.min(img_r))
            img = (img - min_val) / (max_val - min_val)

            print(img.min(), img.max(), img_r.min(), img_r.max())

            # Plot the original and reconstructed image
            plt.figure(figsize=(12, 4))
            plt.subplot(131)
            plt.imshow(jnp.reshape(img[0], (img_shape)))
            plt.title("Original")
            plt.axis('off')
            plt.subplot(132)
            plt.imshow(jnp.reshape(img_r, (img_shape)))
            plt.title("Reconstructed")
            plt.axis('off')

            # Plot the poses
            plt.subplot(133)
            plt.imshow(jnp.reshape(img_r, (img_shape)))
            plt.title("Poses")
            plt.axis('off')

            # Poses are [-1, 1], map to [0, img_shape]
            poses_m = (p_b + 1) / 2 * img_shape[0]
            plt.scatter(poses_m[0, :, 0], poses_m[0, :, 1], c='r')
            plt.close('all')
            # Save checkpoint if lowest loss
            if sum(epoch_loss) / len(epoch_loss) < lowest_loss:
                lowest_loss = sum(epoch_loss) / len(epoch_loss)

                # checkpoint_manager.save(step=epoch, args=ocp.args.Composite(
                #     state=ocp.args.StandardSave(enf_params),
                #     config=ocp.args.JsonSave(config.to_dict())))


/gpfs/home1/rvalperga/miniforge3/envs/enf/lib/python3.11/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


/gpfs/home1/rvalperga/miniforge3/envs/enf/lib/python3.11/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/tmp/ipykernel_2702159/4013305820.py:2: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  return np.array(image, dtype=np.float32) / 255
/tmp/ipykernel_2702159/4013305820.py:2: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-

KeyboardInterrupt: 